In [1]:
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
client = OpenAI(
    api_key=os.getenv("RUNPOD_TOKEN"),
    base_url=os.getenv("RUNPOD_CHATBOT_URL"),
)
model_name = os.getenv("MODEL_NAME")

In [3]:
def get_chatbot_response(client, model_name, message, temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=0.0,
        max_tokens=2000,
        top_p=0.8,
        
    ).choices[0].message.content
    return response

# Get LLM response 

In [4]:
messages = [{"role": "system", "content": "What's the capital of Germany?"}]
response = get_chatbot_response(client, model_name, messages)

In [5]:
response

'The capital of Germany is Berlin.'

# Prompt Engineering

## Structured Output

In [6]:
system_prompt = """
You are a helpful assistant that answer questions about capitals of countries.

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
[
{
    "country": the country that you will get the capital of 
    "capital": the capital of the country stated
}
]
"""
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':"What's the capital of Germany?"})
response = get_chatbot_response(client,model_name,messages)
print(response)

[
{
    "country": "Germany",
    "capital": "Berlin"
}
]


In [7]:
type(response)

str

In [8]:
json_response = json.loads(response)
json_response

[{'country': 'Germany', 'capital': 'Berlin'}]

In [9]:
type(json_response[0]), json_response[0]['capital']

(dict, 'Berlin')

## Input Structuring

In [29]:
user_input = """
Get me the capital of the following countries:
'''

1. Italy
2. France
3. Germany
'''

"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": user_input})
response = get_chatbot_response(client, model_name, messages)
print(response)

[
    {
        "country": "Italy",
        "capital": "Rome"
    },
    {
        "country": "France",
        "capital": "Paris"
    },
    {
        "country": "Germany",
        "capital": "Berlin"
    }
]


In [30]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'},
 {'country': 'France', 'capital': 'Paris'},
 {'country': 'Germany', 'capital': 'Berlin'}]

## Give the model time to think (Chain of Thought)

In [31]:
user_prompt = """
Calculate the result of this equation : 1+3

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{
    result : The final number resulted from calculating the result equation above
}
"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

{
    "result" : 4
}


# RAG - Retrieval Augmented Generation

In [32]:
user_prompt = """
What's new in iphone 16 ?
"""

messages = [{"role": 'user', 'content': user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

There is no such thing as an "iPhone 16". Apple has not released an iPhone with that model number. The latest iPhone models available in the market are iPhone 13 series, which includes iPhone 13, iPhone 13 Pro, and iPhone 13 Pro Max.

However, if you're looking for information on the rumored features of the next-generation iPhone, which is expected to be called iPhone 14, here are some of the rumored features:

* Improved cameras: The iPhone 14 is expected to feature a new triple-camera setup with a wider-angle lens, a telephoto lens, and a ultra-wide-angle lens.
* Faster processor: The iPhone 14 is expected to be powered by Apple's A15 Bionic chip, which is expected to provide a significant boost in performance and efficiency.
* Longer battery life: The iPhone 14 is expected to have a longer battery life, with some reports suggesting up to 12 hours of internet use on a single charge.
* New design: The iPhone 14 is expected to have a new design, with a slightly larger screen and a more

In [14]:
pip show openai

Name: openai
Version: 1.50.2
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: d:\conda\envs\env\lib\site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.
